In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
train_dir = r"C:\Users\LENOVO\OneDrive\Desktop\Plant disease detection\PlantVillageDataset\train_val_test\train"
val_dir   = r"C:\Users\LENOVO\OneDrive\Desktop\Plant disease detection\PlantVillageDataset\train_val_test/val"
test_dir  = r"C:\Users\LENOVO\OneDrive\Desktop\Plant disease detection\PlantVillageDataset\train_val_test\test"

In [3]:
Img_size= (128, 128)
Batch_size= 32
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=Img_size,
    batch_size=Batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=Img_size,
    batch_size=Batch_size
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=Img_size,
    batch_size=Batch_size
    
)

Found 14437 files belonging to 15 classes.
Found 2073 files belonging to 15 classes.
Found 4128 files belonging to 15 classes.


In [4]:
class_names = train_ds.class_names
print(class_names)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [5]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [6]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

In [9]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,394,255 (28.21 MB)

 Trainable params: 7,394,255 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 147s 260ms/step - accuracy: 0.6290 - loss: 1.1833 - val_accuracy: 0.7916 - val_loss: 0.6464
Epoch 2/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 114s 252ms/step - accuracy: 0.8445 - loss: 0.4658 - val_accuracy: 0.8080 - val_loss: 0.5378
Epoch 3/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 117s 259ms/step - accuracy: 0.9191 - loss: 0.2461 - val_accuracy: 0.8167 - val_loss: 0.5436
Epoch 4/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 138s 249ms/step - accuracy: 0.9529 - loss: 0.1407 - val_accuracy: 0.8548 - val_loss: 0.5213
Epoch 5/5
452/452 ━━━━━━━━━━━━━━━━━━━━ 110s 242ms/step - accuracy: 0.9736 - loss: 0.0854 - val_accuracy: 0.8423 - val_loss: 0.6200


In [12]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)

129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 105ms/step - accuracy: 0.8447 - loss: 0.6263
Test Accuracy: 0.8447189927101135


In [14]:
model.save("plant_disease_model.keras")